# MIMIC 4 data - dataset construction admissions

Code taken from GRU-ODE-Bayes preprocessing; simplified and adapted for MIMIC 4 1.0

In [7]:
import os
import pathlib
import pandas as pd

p_project = str(pathlib.Path(os.getcwd()).parents[1])

In [8]:
path_data = p_project + '/data/mimic4'

In [9]:
fn = path_data + '/raw/hosp/admissions.csv.gz'
adm = pd.read_csv(fn, compression='gzip')

In [10]:
#keep only patients present in patients data
patients_df=pd.read_csv(path_data + '/raw/hosp/patients.csv.gz')
adm_dob=pd.merge(patients_df[['subject_id','anchor_age']], adm, on='subject_id')

In [11]:
#keep only patients present in ICU
icustay_df = pd.read_csv(path_data + '/raw/icu/icustays.csv.gz', compression='gzip')
adm_icu = pd.merge(icustay_df[['hadm_id', 'los']], adm_dob, on='hadm_id')
adm_icu.rename(columns = {'los': 'icu_los'}, inplace = True)

In [12]:
adm_icu_selected=adm_icu.loc[(adm_icu['icu_los']<30) & (adm_icu['icu_los']>1)]
print('Number of patients remaining in the dataframe: ')
print(len(adm_icu_selected.index))

Number of patients remaining in the dataframe: 
57309


In [13]:
adm_icu_selected.head()

,hadm_id,icu_los,subject_id,anchor_age,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
2,24597018,1.118032,10001217,55,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0
4,25563031,1.338588,10001725,46,2110-04-11 15:08:00,2110-04-14 15:00:00,NaN,EW EMER.,P35SU0,PACU,HOME,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,0
5,26184834,9.171817,10001884,68,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,OBSERVATION ADMIT,P874LG,EMERGENCY ROOM,DIED,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2131-01-07 13:36:00,2131-01-07 22:13:00,1
6,23581541,1.314352,10002013,53,2160-05-18 07:45:00,2160-05-23 13:30:00,NaN,SURGICAL SAME DAY ADMISSION,P47E1G,PHYSICIAN REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,SINGLE,OTHER,NaN,NaN,0
8,23822395,6.178912,10002155,80,2129-08-04 12:44:00,2129-08-18 16:53:00,NaN,EW EMER.,P05HUO,PROCEDURE SITE,CHRONIC/LONG TERM ACUTE CARE,Other,ENGLISH,MARRIED,WHITE,2129-08-04 11:00:00,2129-08-04 12:35:00,0


In [14]:
path_processed = p_project + '/data/mimic4/processed'

if not os.path.exists(path_processed):
    os.mkdir(path_processed)

In [15]:
adm_icu_selected.to_csv(path_processed + '/admissions_processed.csv')